## Breast Cancer Prediction using the Wisconsin Cancer Dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The dataset used is the very popular Wisconsin Cancer dataset. The link to the data is - https://www.kaggle.com/uciml/breast-cancer-wisconsin-data.
The data has been licensed under CC BY-NC-SA 4.0. Details about the license can be found at - https://creativecommons.org/licenses/by-nc-sa/4.0/.

The author of the dataset, in no way endorses this project of mine.

### Reading the data

In [ ]:
df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')
#The data has a column called  'Unnamed: 32' with nothing but NaN values.
df.drop(['Unnamed: 32'], axis = 1, inplace = True)
df.head()

### Performing preliminary EDA

In [ ]:
print(df.isnull().sum().sum())
print(df.shape)

The data has 569 rows of patient data with 32 features. The data also has no missing values.

### Identifying target variable and encoding it to support Classification.
The target variable is the 'diagnosis' column which classifies the cancerous tumour as Malignant(M) or Benign(B). Malignant tumours are cancerous whereas Benign aren't. 


In [ ]:
df['diagnosis'].replace({'B' : 0, 'M' : 1}, inplace = True)
df['diagnosis'].value_counts()

### Splitting the data into train and test set.

The test set is for final check of the accuracy metric used. Validation of the data will be done on the test data itself using cross validation.  

In [ ]:
# Allocating 69 rows to the test data and rest to the train data. 
df_train = df.iloc[:500]
df_test = df.iloc[500:]

# Re-indexing of the test data
df_test.reset_index(inplace = True)
df_test.drop('index', axis = 1,inplace = True)

### Feature Analysis

Using co-relation matrix to identify important features

In [ ]:
df_corr = df_train.corr()
df_corr = df_corr.where(abs(df_corr['diagnosis']) > 0.5)
df_corr.dropna(inplace=True)
cols_to_drop = df_corr.columns.drop(df_corr.index.drop('diagnosis'))
df_corr.drop(columns = cols_to_drop, inplace = True)
print("Important features  are - ", df_corr.columns.values)

**Note - It is important to note that in Regression and Classification models, the features should be independent of each other. This ensures better fitting and less chances of errors.** 

Hence only one feature is selcted from each set of mean,se and worst values.

In [ ]:
#defining the features
features = ['radius_mean', 'concave points_mean', 'radius_se', 'radius_worst', 'concave points_worst']

### Normalizing the data

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df_train[features]
y = df_train.diagnosis
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_test = df_test[features]
y_test = df_test.diagnosis
scale = StandardScaler()
X_test_scaled = scale.fit_transform(X_test)

### Classification model(s)

* Random Forest Classification and hyperparameter tuning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

#developing baseline model
RF_model = RandomForestClassifier(n_estimators = 1000, max_depth = 10, random_state = 7, n_jobs = -1)
scores = cross_val_score(RF_model, X_scaled, y, cv = 5, scoring = 'f1')
scores.mean()*100

This code snippet tunes 20 values. Hence it might take a few mins to run.... The optimized hyper-paramters will be displayed on the next cell... 

In [ ]:
#hyperparamter tuning
for n_est in [200,400,600,800]:
    for m_d in [5,6,7,8,12]:
        RF_model = RandomForestClassifier(n_estimators = n_est, max_depth = m_d, random_state = 7, n_jobs = -1)
        scores = cross_val_score(RF_model, X_scaled, y, cv = 5, scoring = 'f1')
        print('estimators = ',n_est," max depth = ",m_d," score = ",scores.mean()*100)

Optimized Hyperparamters are - 600 and 10 respectively.

In [ ]:
#developing the optimized model
RF_model = RandomForestClassifier(n_estimators = 600, max_depth = 10, random_state = 7, n_jobs = -1)
RF_model.fit(X,y)
scores = cross_val_score(RF_model, X_scaled, y, cv = 5, scoring = 'f1')
scores.mean()*100

With a tuned RandomForest ensemble model, we get a f1 score of 92.26%

* K-Nearest Neighbour Classification and hyperparameter tuning

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

#developing baseline model
knn_model = KNeighborsClassifier(n_neighbors = 5, leaf_size = 20, n_jobs = -1)
scores = cross_val_score(knn_model, X_scaled, y, cv = 5, scoring = 'f1')
scores.mean()*100

In [ ]:
#hyperparameter tuning
for ne in [4,5,6,7,8,9,10]:
    for ls in [10,20,30,40,50,60,80]:
        knn_model = KNeighborsClassifier(n_neighbors = ne, leaf_size = ls, n_jobs = -1)
        scores = cross_val_score(knn_model, X_scaled, y, cv = 5, scoring = 'f1')
        print('n_neighbors = ', ne, ' leaf_size = ', ls, 'score = ', scores.mean()*100)            

Tuned hyperparameters are - 5 and 20 respectively, which is the baseline model itself.

In [ ]:
#developing the optimized model
knn_model = KNeighborsClassifier(n_neighbors = 5, leaf_size = 20, n_jobs = -1)
scores = cross_val_score(knn_model, X_scaled, y, cv = 5, scoring = 'f1')
scores.mean()*100

With a tuned KNN model we get an f1 score of 92.25%.

* Logistic Regression and hyperparamter tuning

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

#developing the baseline model and tuning it
for c in [0.1,0.2,0.3,0.4,0.5,1]:
    LoReg_model = LogisticRegression(C = c, random_state = 3, n_jobs = -1)
    scores = cross_val_score(LoReg_model, X_scaled ,y , cv=5, scoring='f1')
    print(scores.mean()*100)

The tuned hyperparameter is C = 0.5

In [ ]:
#developing the optimized model
LoReg_model = LogisticRegression(C = 0.5, random_state = 3, n_jobs = -1)
scores = cross_val_score(LoReg_model, X_scaled ,y , cv=5, scoring='f1')
scores.mean()*100

With a slightly tuned Logistic Regression Classifier, we get a f1_score of 92.45%

From the above tuned models, it can be seen that, all of the models have a similar f1 score around 92. The f1_score is higher because of the earlier feature selection we performed. To finalize the model, I will be using the Random Forest Classifer. This is because, Random Forest is an ensemble classifier and it generally works better with un-seen data i.e. test data.

### Analysis on Test Data (with yellowbrick)

In [ ]:
#Fit the final model on the train data
final_model = RandomForestClassifier(n_estimators = 600, max_depth = 10, random_state = 7, n_jobs = -1)
final_model.fit(X_scaled, y);

In [ ]:
#Scoring on test data
y_true = y_test
y_pred = final_model.predict(X_test_scaled)

In [ ]:
#defining labels for prognosis
labels = ['Benign', 'Malignant']

#using yellowbricks Classification Report
from yellowbrick.classifier import ClassificationReport

report = ClassificationReport(final_model, size=(480,240), classes = labels, cmap = 'PuBu')
report.score(X_test_scaled, y_test)
report.show()  

In [ ]:
#using yellowbrick class prediction error
from yellowbrick.classifier import ClassPredictionError

error = ClassPredictionError(final_model, size=(540,360), classes = labels)
error.score(X_test_scaled, y_test)
error.show()

### Inferences and Report on the Test data

The model has been trained on a data of 500 rows and tested on a smaller data of 69 rows. From the yellowbrick analysis of test data, it can be ascertained that the model has performed well with the previously unseen test data. Owing to the small size of the test data, the following results can't be generalized.

For the class label Benign, i.e. non cancerous tumours, the model always predicts correctly and hence has a precision of 100%. However, for the class label Malignant, i.e. actual cancerous tumours, the model most of the time predicts correctly. It has a precision of 0.8%. 

------------------